In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from natsort import natsorted
import numpy as np
import matplotlib.animation as animation
import xarray as xr
import imageio
import matplotlib
import sys
import os

# Assuming /AutomationModule is in the root directory of your project
sys.path.append(os.path.abspath(r'H:\phd stuff\tidy3d'))

from AutomationModule import * 

import AutomationModule as AM

In [ ]:
tidy3dAPI = os.environ["API_TIDY3D_KEY"]

In [ ]:
a = 1
medium = td.Medium(permittivity=11.56)
run = True

In [ ]:
# file = r"h:\phd stuff\structures\SHU_2D\chi_0.20_N484_posics.dat"
file = r"h:\phd stuff\structures\SHU_2D\chi_0.45_N484_posics.dat"
data = []
centers = []
slicing= 1
# slicing= 0.1

with open(file,"r") as f: 
    lines = f.read().splitlines() 
    i=0
    index = 1
    num_samples = int(lines[index].split()[0])

    while num_samples>0 and i < 20:
        try:
            num_samples = int(lines[index].split()[0])
            info = lines[index]
            old_index = index
            index += num_samples+1
            li = (lines[old_index+1:index])
            centers = np.array([list(map(float, item.split())) for item in li])
            data += [{
                "centers":centers,
                "Lx":float(lines[old_index].split()[1]),
                "Ly":float(lines[old_index].split()[2]),
                "points":float(lines[old_index].split()[0]),
            }]
            i+=1
        except:
            num_samples = 0

In [ ]:
# lambdas =  a/np.array([0.1,0.12]) #Diffusive
# lambdas =  a/np.array([0.12,0.15]) #Diffusive
# lambdas =  a/np.array([0.144,0.18]) #Diffusive
lambdas =  a/np.array([0.28,0.48]) #SAL

In [ ]:
sample = 10
project_name = r"08_23_2024 chi 0.2 N484 Gap Good One"
# project_name = r"08_22_2024 chi 0.45 N484 T(t)"
empty=False

runtime_ps = 40e-12
min_steps_per_lambda = 20
for item in [a/np.array([0.12,0.51])]:
 lambdas = item
 add_ref = True
 for k,item in enumerate(data):
    if k > sample:
        continue


    structure_1 = AM.loadAndRunStructure(key = tidy3dAPI
                ,direction="z", lambda_range=lambdas,
                box_size= data[k]['Lx']*a,runtime_ps=runtime_ps,min_steps_per_lambda=min_steps_per_lambda,
               scaling=1,shuoff_condtion=1e-20, verbose=True,
               monitors=["flux"],
               freqs=100, 
               source="planewave", 
               width=0.20, ref_only=True
               )
           
    # print(structure_1.__str__())

    sim = structure_1.sim



    boundaries= td.BoundarySpec(
            x=td.Boundary.periodic(),
            y=td.Boundary(plus=td.Absorber(num_layers=500),minus=td.Absorber(num_layers=500)) ,
            z=td.Boundary.periodic(),
        )
    sim = sim.copy(update={'boundary_spec':boundaries})
    sim = sim.copy(update={'sources':[]})
    sim = sim.copy(update={'monitors':[]})


    Lx, Ly = data[k]['Lx'],30
    sim = sim.copy(update={'size':[Lx,Ly,0]})

    source = td.PlaneWave(
            source_time = td.GaussianPulse(
                freq0=structure_1.freq0,
                fwidth=structure_1.freqw
            ),
            size= (td.inf,0,td.inf),
            center=(0,-14,0),
            direction='+',
            pol_angle=np.pi/2,
            name='planewave',
            )

    
    # ax1 = source.source_time.plot(times=np.linspace(0,structure_1.t_stop, 10000))

    # # Extract the data from the Axes object
    # line = ax1.get_lines()[0]
    # times = line.get_xdata()
    # amplitude = line.get_ydata()

    # fig, ax = plt.subplots(figsize=(15, 10))
    # ax.plot(times*1e12, amplitude)
    # ax.set_xlabel(r"t[ps]")
    # ax.set_ylabel('Amplitude')
    # ax.set_title('Source Amplitude')
    # ax.legend(['Source Spectrum'])
    # plt.xlim(0,1)
    # #plt.savefig(f'{store_path}/source_time.pdf', format='pdf')
    # plt.show()
    
    # ax2=source.source_time.plot_spectrum(times=np.linspace(0, structure_1.t_stop, 10000))
    #  #Extract the data from the Axes object
    # c = td.C_0 
    # line = ax2.get_lines()[0]
    # freqs = line.get_xdata()
    # amplitude = line.get_ydata()

    # # Convert frequency to wavelength (in meters)
    # wavelengths = c / freqs

    # fig, ax = plt.subplots(figsize=(15, 10))
    # ax.plot(a/wavelengths, amplitude)
    # ax.set_xlabel(r"$\nu$'")
    # ax.set_ylabel('Amplitude')
    # ax.set_title('Source Spectrum')
    # ax.legend(['Source Spectrum'])
    # #plt.savefig(f'{store_path}/source_freqs.pdf', format='pdf')
    # plt.show()
    
    sim = sim.copy(update={'sources':[source]})

    time_monitorFieldOut = td.FieldTimeMonitor(
                center = (0,0,0),
                size = (Lx,Ly,0),
                    start=0,
                    stop=60e-12,
                    interval=50,
                    fields=["Ex", "Ey", "Ez"],
                    name="time_monitorFieldOut",
                    
                )
    

    freq_monitorFieldOut = td.FieldMonitor(
                center = (0,0,0),
                size = (td.inf, Ly,0),
                   freqs =structure_1.monitor_freqs,
                    fields=["Ex", "Ey", "Ez"],
                    name="freq_monitorFieldOut",
                    
                )

    
    time_monitorT = td.FluxTimeMonitor(
                    center = (
                            0,12.5,0
                            ),
                size = (
                   td.inf,0,td.inf
                    ),
                    interval = 200,
                    name="time_monitorT",

                )
    
    monitor_1 = td.FluxMonitor(
                center = (
                            0,-12.5,0
                            ),
                size = (
                   td.inf,0,td.inf
                    ),
                freqs = structure_1.monitor_freqs,
                name='flux2' )

    monitor_2 = td.FluxMonitor(
                center = (
                            0,12.5,0
                            ),
                size = (
                   td.inf,0,td.inf
                    ),
                freqs = structure_1.monitor_freqs,
                name='flux1' 
            )
    
    
    sim = sim.copy(update={'monitors':[monitor_1,monitor_2]})
    
    slab_1 = td.Structure(
                    geometry=td.Box(
                        center= (0,(Ly/2+Lx*slicing/2)/2,0),
                        size=(td.inf,(Ly/2-Lx*slicing/2),td.inf),
                    ),
                    
                    medium=medium,
                    name='slab1',
                    )

    slab_2 = td.Structure(
                    geometry=td.Box(
                        center= (0,-(Ly/2+Lx*slicing/2)/2,0),
                        size=(td.inf,(Ly/2-Lx*slicing/2),td.inf),
                    ),
                    medium=medium,
                    name='slab2',
                    )
    
    cyl_group = []
    for x,y in data[k]['centers']:
        if np.abs(y)<=(Lx/2)*slicing:
            cyl_group.append(td.Cylinder(center=[x, y, 0], radius=0.189, length=td.inf))

    cylinders = td.Structure(geometry=td.GeometryGroup(geometries=cyl_group), medium=medium)


    if empty:
        sim = sim.copy(update={'structures':[]})
    else:
        sim = sim.copy(update={'structures':[cylinders],"grid_spec": td.GridSpec.uniform(dl=structure_1.dl)})
    # fig, ax = plt.subplots(1, tight_layout=True, figsize=(16, 8))
    # sim.plot(z=0, ax=ax)
    # plt.show()

    sim_name = f"pnas_T_t_Localization {1/lambdas[0]:.3g} - {1/lambdas[1]:.3g} - Sample_{str(k)}"
    print(sim_name)
    print(sim.num_time_steps)
    if run:

        if add_ref:
            print("running ref...")
            id0 = web.upload(sim.copy(update={"structures":[]}), folder_name=project_name,task_name=sim_name+'_0', verbose=True)
            web.start(task_id = id0)
            web.monitor(task_id=id0,verbose=True)
            add_ref=False


        id =web.upload(sim, folder_name=project_name,task_name=sim_name, verbose=True)
        web.start(task_id = id)
        web.monitor(task_id=id,verbose=True)
    
        ids = (id0 if id0 else '') +'\n' + id
        incidence_folder = "z_incidence"
        file_path = rf"H:\phd stuff\tidy3d\data/{project_name}/{incidence_folder}/{sim_name}.txt"
        # Check if the folder exists
        if not os.path.exists( rf"H:\phd stuff\tidy3d\data/{project_name}/{incidence_folder}"):
            os.makedirs(rf"H:\phd stuff\tidy3d\data/{project_name}/{incidence_folder}")
            print(f"Folder '{project_name}/{incidence_folder}' created successfully.")

        # Open file in write mode
        with open(file_path, "w") as file:
            # Write the string to the file
            file.write(ids)


